In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom
from pymedphys_dicom.dicom import dicom_dose_interpolate

In [ ]:
energy = '06FFF'

In [ ]:
ROOT_DIR = Path(r"S:\Physics\RCCC Specific Files\Dosimetry\Film\MLCs\Transmission")

OPEN_DICOM_DIR = (ROOT_DIR.joinpath(r"DICOM Exports")).joinpath(energy+"_10x10")
CLOSED_DICOM_DIR = (ROOT_DIR.joinpath(r"DICOM Exports")).joinpath(energy+"_Closed")

RESULTS = ROOT_DIR.joinpath(r"Results")

In [ ]:
OPEN_FILM_RESULTS_FILE = (RESULTS.joinpath(r"Data_Open.xlsx"))
open_film_results = pd.read_excel(OPEN_FILM_RESULTS_FILE, sheet_name=energy+'_export').dropna()

CLOSED_FILM_RESULTS_FILE = (RESULTS.joinpath(r"Data_Closed.xlsx"))
closed_film_results = pd.read_excel(CLOSED_FILM_RESULTS_FILE, sheet_name=energy+'_export').dropna()

In [ ]:
open_film_dose = open_film_results.Dose.values
open_film_z = open_film_results.z.values

closed_film_dose = closed_film_results.Dose.values
closed_film_z = closed_film_results.z.values

In [ ]:
open_dosecheck_dose = pydicom.read_file(str(OPEN_DICOM_DIR.joinpath('DoseCHECKCalculatedDose\dose.dcm')), force=True)
open_monaco_dose = pydicom.read_file(str(OPEN_DICOM_DIR.joinpath('RTDose\dose.dcm')), force=True)
open_dicom_plan = pydicom.read_file(str(OPEN_DICOM_DIR.joinpath('RTPlan\plan.dcm')), force=True)

closed_dosecheck_dose = pydicom.read_file(str(CLOSED_DICOM_DIR.joinpath('DoseCHECKCalculatedDose\dose.dcm')), force=True)
closed_monaco_dose = pydicom.read_file(str(CLOSED_DICOM_DIR.joinpath('RTDose\dose.dcm')), force=True)
closed_dicom_plan = pydicom.read_file(str(CLOSED_DICOM_DIR.joinpath('RTPlan\plan.dcm')), force=True)

In [ ]:
depth_maximum = {
    '06MV': -66,
    '10MV': -64,
    '06FFF': -59,
    '10FFF': -56
}

interp_x = [0]
open_interp_y = [depth_maximum[energy]]
closed_interp_y = [-69]

open_interp_coords = (open_film_z, open_interp_y, interp_x)
closed_interp_coords = (closed_film_z, closed_interp_y, interp_x)

In [ ]:
open_monaco_profile = np.squeeze(dicom_dose_interpolate(open_interp_coords, open_monaco_dose))
open_dosecheck_profile = np.squeeze(dicom_dose_interpolate(open_interp_coords, open_dosecheck_dose))

closed_monaco_profile = np.squeeze(dicom_dose_interpolate(closed_interp_coords, closed_monaco_dose))
closed_dosecheck_profile = np.squeeze(dicom_dose_interpolate(closed_interp_coords, closed_dosecheck_dose))

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(open_film_z,open_monaco_profile,label='Monaco')
plt.plot(open_film_z,open_dosecheck_profile,label='DoseCHECK')
plt.plot(open_film_z,open_film_dose*1.05,label='Film')
plt.ylabel('Dose (Gy)')
plt.xlabel('Plane Position (mm)')
plt.title('')
plt.legend(loc='lower left')


plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)

# plt.suptitle(('MLC Test Fields, ' + energy + ' ' + test_field + ' , Doses for Monaco ' + algorithm_full + ', DoseCHECK, and film'),fontsize="x-large",x=1,y=3.2)

# plt.savefig(RESULTS.joinpath(filename + f'_1.png'),bbox_inches='tight')

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(closed_film_z,closed_monaco_profile,label='Monaco')
plt.plot(closed_film_z,closed_dosecheck_profile,label='DoseCHECK')
plt.plot(closed_film_z,closed_film_dose*1.05,label='Film')
plt.ylabel('Dose (Gy)')
plt.xlabel('Plane Position (mm)')
plt.title('')
plt.legend(loc='lower left')
plt.ylim([0,None])

plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)
# plt.suptitle(('MLC Test Fields, ' + energy + ' ' + test_field + ' , Doses for Monaco ' + algorithm_full + ', DoseCHECK, and film'),fontsize="x-large",x=1,y=3.2)

plt.suptitle(('MLC Transmission, ' + energy + ', Monaco, DoseCHECK, and film dose for 5000 MU'),fontsize="x-large",x=1,y=3.2)

plt.savefig(RESULTS.joinpath('Transmission_' + energy + f'_20190927_model.png'),bbox_inches='tight')

In [ ]:
[np.median(closed_film_dose)/50,np.median(closed_monaco_profile)/50,np.median(closed_dosecheck_profile)/50]

In [ ]:
current_leakage_monaco_value = {
    '06MV': 0.005,
    '10MV': 0.003,
    '06FFF': 0.005,
    '10FFF': 0.005
}

In [ ]:
unaccounted_for_monaco_factor = np.median(closed_monaco_profile)/50 / current_leakage_monaco_value[energy]

In [ ]:
np.median(closed_film_dose)/50 / unaccounted_for_monaco_factor

In [ ]:
np.median(closed_film_dose)

In [ ]:
100*((np.median(closed_monaco_profile)/50)/np.median(open_monaco_profile))

In [ ]:
100*((np.median(closed_dosecheck_profile)/50)/np.median(open_dosecheck_profile))

In [ ]:
np.max(open_film_dose)*1.05